# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d
from scipy.stats import skew
from scipy.signal import welch
from scipy.stats import entropy
import warnings
warnings.filterwarnings('ignore')

# Data and Labels

In [2]:
# Define the base directory containing the processed data
base_dir = 'Demo Processed'

# Define the list of conditions (e.g., Healthy and Non Healthy)
conditions = ['Healthy', 'Non Healthy']

# Define the list of tasks for each condition
tasks = {
    'Healthy': ['Healthy Eyes Close', 'Healthy Eyes Open', 'Healthy Task'],
    'Non Healthy': ['MDD Eyes Close', 'MDD Eyes Open', 'MDD Task']
}

# Initialize lists to store epochs data, labels, and group information
epochs_data = []
labels = []
group = []

# Iterate over conditions and tasks to load epochs data
for condition in conditions:
    for task in tasks[condition]:
        # Construct the path to the epochs data directory
        epochs_dir = os.path.join(base_dir, condition, task)
        # Check if the directory exists
        if os.path.isdir(epochs_dir):
            # List files in the directory with _epo.fif suffix
            epochs_files = [os.path.join(epochs_dir, f) for f in os.listdir(epochs_dir) if f.endswith('_epo.fif')]
            # Load epochs data from each file
            for epoch_file in epochs_files:
                epochs = mne.read_epochs(epoch_file)
                # Pick EEG channels
                epochs.pick_types(eeg=True)
                # Append epochs data, labels, and group information
                epochs_data.append(epochs)
                labels.append(epochs.events[:, 2])
                group.extend([len(labels) - 1] * len(epochs))
        else:
            print(f"Directory not found: {epochs_dir}")

# Combine epochs data, labels, and group information
X = mne.concatenate_epochs(epochs_data)
Y = np.hstack(labels)
group = np.array(group)
# Now you can use X, Y, and group for further analysis
# Print dimensions
print("X shape:", X.get_data().shape)
print("Y shape:", Y.shape)

# Now you have concatenated "Eyes Close" epochs data in X and corresponding labels in Y

Reading C:\Users\khand\MDD\W.out Sep (train)\New Workflow\Preprocessing\Feature Extract\Feature Extraction Channelwise\Demo Processed\Healthy\Healthy Eyes Close\H S1 EC_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
299 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Reading C:\Users\khand\MDD\W.out Sep (train)\New Workflow\Preprocessing\Feature Extract\Feature Extraction Channelwise\Demo Processed\Healthy\Healthy Eyes Close\H S10 EC_epo.fif ...
Isotrak not found
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
375 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(.

In [3]:
Y=Y
groups_array = np.hstack(group)

# Functions of Feature Extraction

In [4]:
import numpy as np
import os
import mne
import pywt
from scipy.stats import skew, kurtosis
from scipy.signal import welch, find_peaks
from scipy.stats import entropy

# Your channel mapping dictionary
channel_mapping = {
    'EEG Fp1-LE': 'Fp1',
    'EEG F3-LE': 'F3',
    'EEG C3-LE': 'C3',
    'EEG P3-LE': 'P3',
    'EEG O1-LE': 'O1',
    'EEG F7-LE': 'F7',
    'EEG T3-LE': 'T3',
    'EEG T5-LE': 'T5',
    'EEG Fz-LE': 'Fz',
    'EEG Fp2-LE': 'Fp2',
    'EEG F4-LE': 'F4',
    'EEG C4-LE': 'C4',
    'EEG P4-LE': 'P4',
    'EEG O2-LE': 'O2',
    'EEG F8-LE': 'F8',
    'EEG T4-LE': 'T4',
    'EEG T6-LE': 'T6',
    'EEG Cz-LE': 'Cz',
    'EEG Pz-LE': 'Pz',
    'EEG A2-A1': 'A2'
}

## Time Domain Features

In [5]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

# Function to extract time-domain features from a specific channel
def extract_time_domain_features_single_channel(epochs, channel_index):
    features = []
    for epoch in epochs:
        channel_data = epoch[channel_index].flatten()

        # Basic Time-Domain Features
        mean_val = np.mean(channel_data)
        median_val = np.median(channel_data)
        var_val = np.var(channel_data)
        std_dev = np.std(channel_data)
        skewness = skew(channel_data)
        kurt = kurtosis(channel_data)
        zcr = zero_crossing_rate(channel_data)
        peak_amp = np.ptp(channel_data)

        # Hjorth Parameters
        activity, mobility, complexity = hjorth_parameters(channel_data)

        # Additional Features
        num_waves = len(find_peaks(channel_data)[0])
        wave_duration = len(channel_data) / num_waves if num_waves > 0 else 0

        channel_features = [
            mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
            wave_duration, peak_amp, activity, mobility, complexity
        ]
        features.append(channel_features)

    return np.array(features)

## Frequency Domain Features

In [6]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch



def get_wavelet_coeffs(channel_data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


# Function to extract frequency-domain features from a specific channel
def extract_frequency_domain_features_single_channel(epochs, channel_index, sfreq, wavelet='db4', bands={'delta': (1, 3), 'theta': (4, 7), 'alpha': (8, 12), 'beta': (13, 30), 'gamma': (31, 60), 'sigma': (11, 16)}):
    features = []
    for epoch in epochs:
        channel_data = epoch[channel_index]

        # Compute the Power Spectral Density (PSD)
        freqs, psd = welch(channel_data, sfreq, nperseg=180)

        # Frequency domain features
        mean_val = np.mean(psd)
        median_val = np.median(psd)
        var_val = np.var(psd)
        std_dev = np.std(psd)
        skewness = skew(psd)
        kurt = kurtosis(psd)

        # Compute wavelet coefficients
        wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=3)
        wave_coeffs_mean = np.mean(wave_coeffs[0])

        # Band Power Features
        band_powers = {}
        for band, freq_range in bands.items():
            freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
            band_power = np.sum(psd[freq_mask])
            band_powers[band] = band_power

        # Band Power Ratios
        theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
        beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
        theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']

        # Additional Band Power Ratios
        theta_beta_ratio = band_powers['theta'] / band_powers['beta']
        theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
        gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
        gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])

        channel_features = [
            mean_val, median_val, var_val, std_dev, skewness, kurt,
            band_powers['delta'], band_powers['theta'], band_powers['alpha'],
            band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
            theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,
            theta_beta_ratio, theta_alpha_beta_alpha_ratio, gamma_delta_ratio,
            gamma_beta_delta_alpha_ratio, wave_coeffs_mean
        ]
        features.append(channel_features)

    return np.array(features)

## Descrete Wavelet Transform (DWT) Features

In [7]:
# Function to extract DWT features from a specific channel
def extract_dwt_features_single_channel(epochs, channel_index, wavelet='db4', level=4):
    features = []
    for epoch in epochs:
        channel_data = epoch[channel_index]

        # Perform DWT
        coeffs = pywt.wavedec(channel_data, wavelet, level=level)

        # Concatenate DWT coefficients
        concatenated_coeffs = np.concatenate(coeffs, axis=0)
        features.append(concatenated_coeffs)

    return np.array(features)

# Extract and Save Features

In [12]:
# Iterate over each channel and extract features
for channel_name, channel_label in channel_mapping.items():
    channel_index = X.ch_names.index(channel_name)  # Get the index of the channel
    sfreq=256
    # Extract features for the current channel
    X_time = extract_time_domain_features_single_channel(X.get_data(), channel_index)
    X_frequency = extract_frequency_domain_features_single_channel(X.get_data(), channel_index, sfreq)
    X_dwt = extract_dwt_features_single_channel(X.get_data(), channel_index)

    # Save the features for this channel
    np.savez(f'{channel_label}_time.npz', X_time=X_time)
    np.savez(f'{channel_label}_freq.npz', X_frequency=X_frequency)
    np.savez(f'{channel_label}_dwt.npz', X_dwt=X_dwt)

    print(f"Features for channel {channel_label} saved successfully.")

Features for channel Fp1 saved successfully.
Features for channel F3 saved successfully.
Features for channel C3 saved successfully.
Features for channel P3 saved successfully.
Features for channel O1 saved successfully.
Features for channel F7 saved successfully.
Features for channel T3 saved successfully.
Features for channel T5 saved successfully.
Features for channel Fz saved successfully.
Features for channel Fp2 saved successfully.
Features for channel F4 saved successfully.
Features for channel C4 saved successfully.
Features for channel P4 saved successfully.
Features for channel O2 saved successfully.
Features for channel F8 saved successfully.
Features for channel T4 saved successfully.
Features for channel T6 saved successfully.
Features for channel Cz saved successfully.
Features for channel Pz saved successfully.
Features for channel A2 saved successfully.


# Complete